In [522]:
import cv2
import mediapipe as mp
import utils
import pyautogui
import keyboard
import numpy as np
import time

In [523]:
screen_w, screen_h = pyautogui.size()
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity = 1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands = 1
)

In [524]:
def find_landmark_cordinates(processed,index,frame):
        frame_h, frame_w, _ = frame.shape
        if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                x = int(hand_landmarks.landmark[index].x * frame_w)
                y = int(hand_landmarks.landmark[index].y * frame_h)
                return (x,y)
        return None

In [525]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_w)
        y = int(index_finger_tip.y * screen_h)
        pyautogui.moveTo(x,y)

In [526]:
can_gesture = False
first_cord_captured = False
initial_cord = ()
hand_closed_start_time = None
def detect_gestures(frame,landmarks_list,processed):
    global can_gesture
    global initial_cord
    global first_cord_captured
    global hand_closed_start_time

    min_volume = 30
    max_volume = 300
    max_hand_closed_timer = 1


    if len(landmarks_list) >= 21:
        index_wrist_dist = utils.get_distance([landmarks_list[8],landmarks_list[0]])
        thumb_index_dist = utils.get_distance([landmarks_list[8],landmarks_list[4]])


        current_cord = ()
        if index_wrist_dist > 290:
            can_gesture = True
            hand_closed_start_time = None

            if not first_cord_captured:
                initial_cord = find_landmark_cordinates(processed,8,frame)            
                first_cord_captured = True
            else:
                current_cord = find_landmark_cordinates(processed,8,frame)
        else:
            if hand_closed_start_time is None:
                hand_closed_start_time = time.time()
            elif (time.time() - hand_closed_start_time) > max_hand_closed_timer: 
                can_gesture = False
                first_cord_captured = False

        frame_h, frame_w, _ = frame.shape
        string = "IW:{0} TI:{1}".format(str(int(index_wrist_dist)),str(int(thumb_index_dist)))
        cv2.putText(frame, string, (int(frame_w / 2) - 200, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (167,191,62), 3)

        #Draw ready trinagle on initial cordinate 
        if first_cord_captured:
            cx, cy = initial_cord
            size = 30
            half_size = size / 2
            color = (210,0,255)
            thickness = 2
            pt1 = (int(cx), int(cy - half_size))  # Top
            pt2 = (int(cx - half_size), int(cy + half_size))  # Bottom left
            pt3 = (int(cx + half_size), int(cy + half_size))  # Bottom right
            triangle_cnt = np.array([pt1, pt2, pt3])
            cv2.drawContours(frame, [triangle_cnt], 0, color, thickness)
            # cv2.circle(frame, (initial_cord[0],initial_cord[1]), 3, (0,0,0), 2)
        
        if first_cord_captured and current_cord and can_gesture:
            
            if initial_cord[0] > frame_w / 2 and current_cord[0] < frame_w / 2:
                keyboard.send("next track")
                initial_cord = ()
                first_cord_captured = False
            elif initial_cord[0] < frame_w / 2 and current_cord[0] > frame_w / 2:
                keyboard.send("previous track")
                initial_cord = ()
                first_cord_captured = False




In [527]:
def draw_landmarks(processed,frame):
    index_finger_tip = find_landmark_cordinates(processed,8,frame)
    thumb_finger_tip = find_landmark_cordinates(processed,4,frame)
    wrist = find_landmark_cordinates(processed,0,frame)

    if index_finger_tip:
        cv2.circle(frame, (index_finger_tip[0],index_finger_tip[1]), 10, (0,255,0), 2)
    if thumb_finger_tip:
        cv2.circle(frame, (thumb_finger_tip[0],thumb_finger_tip[1]), 10, (255,125,0), 2)
    if wrist:
        cv2.circle(frame, (wrist[0],wrist[1]), 10, (0,255,255), 2)


    

In [528]:
cap = cv2.VideoCapture(0)
draw = mp.solutions.drawing_utils

try:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_h, frame_w, _ = frame.shape
        
        if not ret:
            break
        frame = cv2.flip(frame,1)
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        processed = hands.process(frameRGB)
        
        landmarks_list = []

        if processed.multi_hand_landmarks:
            hand_landkarms = processed.multi_hand_landmarks[0]
            draw.draw_landmarks(frame, hand_landkarms, mpHands.HAND_CONNECTIONS)

            for lm in hand_landkarms.landmark:
                landmarks_list.append((lm.x,lm.y))
        
        draw_landmarks(processed,frame)
        detect_gestures(frame,landmarks_list,processed)
        
        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
finally:
    cap.release()
    cv2.destroyAllWindows()